In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[7], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

# **Load data:**

In [ ]:
train_data= pd.read_csv('../input/taxi-data/taxi_data/train_hire_stats.csv')

# **Preprcessing:**
#### TURN DATE Training Data int Frequency

In [ ]:
''' Get Training Data time '''
holiday=['2016-02-06','2016-02-07','2016-02-08','2016-02-09','2016-02-10','2016-02-11','2016-02-12','2016-02-13','2016-02-14','2016-02-27','2016-02-28','2016-02-29',
        '2016-04-02','2016-04-03','2016-04-04','2016-04-05','2016-06-09','2016-06-10','2016-06-11','2016-06-12',
        '2016-09-15','2016-09-16','2016-09-17','2016-09-18',
         '2016-10-09', '2016-10-10', '2016-10-11','2016-12-31','2017-01-01','2017-01-02',
         '2017-01-27','2017-01-28','2017-01-29','2017-01-30','2017-01-31',
         '2017-02-01','2017-02-25','2017-02-26','2017-02-27','2017-02-28'
        ]
train_time= pd.to_datetime(train_data['Date'], format='%Y-%m-%d')
train_time_df=pd.DataFrame()
#train_time_df["year"] = time_df_for.dt.year
train_time_df["month"] = train_time.dt.month
train_time_df["day"] = train_time.dt.day
train_time_df["weekday"] = train_time.dt.weekday
train_time_df["workday"] = 0
train_time_df
'''Train_df Data time'''

train_df= pd.DataFrame()
train_time_df["workday"] = 0
for i in train_time_df.index: 
     if train_time_df["weekday"][i]==5 or  train_time_df["weekday"][i]==6 or str(train_time.iloc[i]).split()[0] in holiday :
        train_time_df["workday"][i]=0
     else:
        train_time_df["workday"][i]=1
train_df['month_cos']=np.sin(2*np.pi*train_time_df["month"]/12)
train_df['month_sin']=np.cos(2*np.pi*train_time_df["month"]/12)
train_df['day_cos']=np.cos(2*np.pi*train_time_df["day"]/30)
train_df['day_sin']=np.sin(2*np.pi*train_time_df["day"]/31)
train_df['hour_cos']=np.cos(2*np.pi*train_data["Hour_slot"]/24)
train_df['hour_sine']=np.sin(2*np.pi*train_data["Hour_slot"]/24)
train_df['zone_id']=train_data['Zone_ID']
train_df["workday"]=train_time_df["workday"]
train_df['Hire_count']=train_data['Hire_count']
# train_df['month_cos'][:]
#fig.subplots_adjust(hspace=0.5, wspace=0.001)
fig=plt.figure( figsize=(15, 12), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.005)
# ''' Attention remmber   RUNING THE Tensorflow AFFINE TRANSFORM FIRST OTHERWISE WILL POPOUT ERROR'''
id=1
row=3
col=1
data_all=[train_df['month_cos'][:8786], 
          #train_df['month_sin'][:8786], 
          
            train_df['day_cos'][:750],
            #train_df['day_sin'][:750],

            train_df['hour_cos'][:24],
            #train_df['hour_sine'][:24]
         ]
title=['Month_sine_cos', 'Day_sine_cos', 'Hour_sine_cos']
i_=0
# for i in range(3):
#     if i==1:
#         i=2
#     elif i==2: 
#         i=4
        
#     fig.add_subplot(row, col, id)
    
#     plt.plot(np.array(data_all[i]), scaley=True, scalex=True)
#     plt.plot(np.array(data_all[i+1]),scaley=True, scalex=True)
#     plt.title(title[i_])
#     i_+=1
#     id+=1
#     plt.show()   
    #plt.show()

### Normalize Splitting Data INTO Training and Testing DATA 
**We will using 80% data for training data, 20% for validation**

**1. Ensure that chopping data into windows of consecutive sample is still possible**

**2.Ensure that Training/Validation data will not shuffle because due to the time sereis data --> have time and Position is matter Most**

In [ ]:
train_df

In [ ]:
month_cos_min= np.min(train_df['day_cos'])
month_cos_max= np.max(train_df['day_cos'])
print(month_cos_min)
print(month_cos_max)

In [ ]:
from sklearn.preprocessing import minmax_scale
import pandas as pd
##Normalize Data 
from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder()
enc = OneHotEncoder()

# 2. FIT
enc.fit(train_df['zone_id'].values)

zone_id=enc.transform(train_df['zone_id']).toarray()
train_df['weekday']= minmax_scale(train_df['weekday'],feature_range=(0, 1))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
encoder= LabelEncoder()
#train_df= train_df.drop(['Hire_count'], axis=1)
train_y=pd.DataFrame()
train_y['Hire_count']=encoder.fit_transform(train_df['Hire_count'].values)
train_y=train_y.astype('float32')
train_y
# take the Encode label scale into Certain Range values
scaler=MinMaxScaler(feature_range=(0,1))
train_y_scale=scaler.fit_transform(train_y)## data type pass inn should in the form of (Dataframe)
print(np.max(train_y))
print(np.max(train_y_scale))
## Invert Transform back to the orginal value
true_value= scaler.inverse_transform(train_y_scale)
print(np.max(true_value))

In [ ]:
train_df['Hire_count']= train_y_scale
train_df[['zone_id']]= minmax_scale(train_df[['zone_id']],feature_range=(0, 1))
train_df

In [ ]:
import sklearn
## Here shuffle IS good IDEA for Regression Model
#train_df=sklearn.utils.shuffle(train_df)
n=len(train_df)
training_df= train_df[0:int(n*0.8)]
val_df= train_df[int(n*0.8):]
print(len(training_df))
print(len(val_df))

**Normalize and Preprocessing for Test data**

In [ ]:
test_data= pd.read_csv('../input/taxi-data/taxi_data/test_hire_stats.csv')


'''Removing the Repeat in Time '''

time_df=pd.DataFrame()

# test_time_df['date']= test_df['Date']
# test_time_df['zone_id']=test_df['Zone_ID']
# # print('Unique values of ZoneID:', time_df.zone_id.unique())
# test_time_df_rep = test_time_df.groupby(test_time_df.columns.tolist()).size().reset_index().rename(columns={0:'count'})
# print(test_time_df_rep)

''' Get Training Data time '''
test_time= pd.to_datetime(test_data['Date'], format='%Y-%m-%d')
test_time_df=pd.DataFrame()
#train_time_df["year"] = time_df_for.dt.year
test_time_df["month"] = test_time.dt.month
test_time_df["day"] = test_time.dt.day
test_time_df["weekday"] = test_time.dt.weekday
test_time_df
'''Train_df Data time'''

test_df= pd.DataFrame()
test_df['weekday']=test_time_df['weekday']
test_df['month_cos']=np.sin(2*np.pi*test_time_df["month"]/12)
test_df['month_sin']=np.cos(2*np.pi*test_time_df["month"]/12)
test_df['day_cos']=np.cos(2*np.pi*test_time_df["day"]/30)
test_df['day_sin']=np.sin(2*np.pi*test_time_df["day"]/31)
test_df['hour_cos']=np.cos(2*np.pi*test_data["Hour_slot"]/24)
test_df['hour_sine']=np.sin(2*np.pi*test_data["Hour_slot"]/24)
test_df['zone_id']=test_data['Zone_ID']
test_df['Hire_count']=test_data['Hire_count']
test_df

In [ ]:
zone_id_int=[int(i) for i in (np.linspace(1, 25, 25))]
print(zone_id_int)
# ##Normalize Data 
zone_id= minmax_scale(zone_id_int,feature_range=(0, 1))
zone_id
dict_val= zip(zone_id_int, zone_id)
## GET SCALE FOR zone ID
list_val=[]
for i in dict_val: 
    list_val.append(i)

In [ ]:
## Convert test ZONE_ID the same SCALE
test_zone=test_df['zone_id']
test_zone_id=[]
for i in test_zone: 
    for l in list_val:
        if i==l[0]: 
            #print(l[1])
            test_zone_id.append(l[1])
            
test_zone_id=np.array(test_zone_id)
test_df['zone_id']=test_zone_id

In [ ]:
from sklearn.preprocessing import minmax_scale
import pandas as pd
##Normalize Data 
test_df['weekday']= minmax_scale(test_df['weekday'],feature_range=(0, 1))

In [ ]:
test_df

# **Model For Time Regression Model**

**Preparing Data Time Series for Input**

In [ ]:
train_label=training_df['Hire_count']
train_dff=training_df.drop(['Hire_count'], axis=1)
val_label=val_df['Hire_count']
val_dff=val_df.drop(['Hire_count'], axis=1)
train_dff.shape

In [ ]:
from tensorflow import keras

past = 48 #(48/step= number hour obser) 
future = 12 #(12/step = number prediction)
step = 6
sequence_length = int(past / step)
# the model will be showed data of 4 hours -- predict the next 2 hours
# Calculate the Number of batch that should go through
start=past+future
#end= start+ 175680
batch_size=256



dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    train_dff,
    train_label,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
    
)

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    val_dff,
    val_label,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [ ]:
for batch in dataset_train.take(1):
    inputs, targets = batch
### Taking 28 step input to predict 1 single output 
print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

In [ ]:
def callback_list(output_dir, checkpoint): 
    modelckpt_callback = keras.callbacks.ModelCheckpoint(
        monitor="val_loss",
        filepath=checkpoint,
        verbose=2,
        save_weights_only=True,
        save_best_only=True,
    )

    early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                   patience= 50, mode='min')

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                 factor=np.sqrt(.1),
                                 patience=10,
                                 verbose=1,
                                 mode='auto',
                                 min_delta=0.001,
                                 cooldown=0,
                                 min_lr=0.0000001
                                 )
    ## Using lr rate schedule to growth Learning rate during the certain number of EPochs
    lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 20))

    tensorboard =  tf.keras.callbacks.TensorBoard(
            log_dir=output_dir,
            write_graph=False,
            update_freq='epoch',
        )
  
    callbacks_list=[early_stopping, reducelr, modelckpt_callback, tensorboard]#reducelr, lr_schedule,
    
    return callbacks_list



### Container will store all model Performance 
val_performance = {}

**Desig Normal LSTM Model**

In [ ]:
# design Simple LSTM model

inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(256, return_sequences=True)(inputs)
#lstm_out = keras.layers.LSTM(256, return_sequences=True)(lstm_out)
lstm_out = keras.layers.LSTM(128, return_sequences=True)(lstm_out)
lstm_out = keras.layers.LSTM(32, return_sequences=True)(lstm_out)
#Dense = keras.layers.Dense(64)(lstm_out)

outputs = keras.layers.Dense(1)(lstm_out)
#outputs1=keras.layers.Lambda(lambda x: x * 200)(outputs)

model = keras.Model(inputs=inputs, outputs=outputs)


optimizer = keras.optimizers.SGD(lr=1e-3, momentum=0.9)
model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer= tf.optimizers.Adam(lr=5e-3),
                metrics=[tf.metrics.MeanAbsoluteError()])
model.summary()
#residual_lstm.compile(loss='mae', optimizer='adam', me)
#model.load_weights('Method2_Two_lstm_model.h5')
epochs=200
output_dir='./logs_select_lr_v1/multi_LSTM_redu_lr'
check_point="muli_LSTM_check_best_lr.h5"

import os
# making directory
try:
    os.mkdir(output_dir)
except:
    pass

callbacks_list=callback_list(output_dir, check_point)

history=model.fit_generator(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=callbacks_list,
)


In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-6, 1e-1, 0, 30])

### Bidirectional LSTM Model
**This allow input Sequence both Forward and Backwards Concatenate Both Interpretation**

In [ ]:
from tensorflow.keras import layers

inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)
lstm_out = layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(lstm_out)
lstm_out = layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(lstm_out)
lstm_out = layers.Bidirectional(keras.layers.LSTM(32))(lstm_out)
#Dense = keras.layers.Dense(64)(lstm_out)
outputs = keras.layers.Dense(1)(lstm_out)
#outputs1=keras.layers.Lambda(lambda x: x * 200)(outputs)

Bi_lstm = keras.Model(inputs=inputs, outputs=outputs)

optimizer = keras.optimizers.SGD(lr=1e-3, momentum=0.9)
Bi_lstm.compile(loss=tf.losses.MeanSquaredError(),
                optimizer= tf.optimizers.Adam(lr=5e-3),
                metrics=[tf.metrics.MeanAbsoluteError()])
Bi_lstm.summary()
#residual_lstm.compile(loss='mae', optimizer='adam', me)
#model.load_weights('Method2_Two_lstm_model.h5')
epochs=200
output_dir='./logs_select_lr_v1/multi_Bidirect_LSTM_reduce_lr'
check_point="muli_Bi_LSTM_check.h5"

import os
# making directory
try:
    os.mkdir(output_dir)
except:
    pass

callbacks_list=callback_list(output_dir, check_point)

history1=Bi_lstm.fit_generator(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=callbacks_list,
)

In [ ]:
plt.semilogx(history1.history["lr"], history1.history["loss"])
plt.axis([1e-6, 1e-1, 0, 30])

### Design ConvLSTM model

In [ ]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
conv= layers.Conv1D(filters=125, kernel_size=4,  strides=1, 
                    padding="causal", activation="relu")(inputs)
lstm_out = layers.LSTM(256, return_sequences=True)(conv)
lstm_out = keras.layers.LSTM(128, return_sequences=True)(lstm_out)
# lstm_out = keras.layers.LSTM(128, return_sequences=True)(lstm_out)
lstm_out =keras.layers.LSTM(64, return_sequences=True)(lstm_out)
lstm_out = keras.layers.LSTM(32)(lstm_out)
#Dense = keras.layers.Dense(64)(lstm_out)
outputs = keras.layers.Dense(1)(lstm_out)

#outputs1=keras.layers.Lambda(lambda x: x * 200)(outputs)

conv_lstm = keras.Model(inputs=inputs, outputs=outputs)


optimizer = keras.optimizers.SGD(lr=1e-3, momentum=0.9)
conv_lstm.compile(loss=tf.losses.MeanSquaredError(),
                optimizer= tf.optimizers.Adam(lr=5e-3),
                metrics=[tf.metrics.MeanAbsoluteError()])
conv_lstm.summary()

epochs=200
output_dir='./logs_select_lr/multi_Conv_LSTM'
check_point="muli_Conv_LSTM_check.h5"

import os
# making directory
try:
    os.mkdir(output_dir)
except:
    pass

callbacks_list=callback_list(output_dir, check_point)

history2=conv_lstm.fit_generator(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=callbacks_list,
)

In [ ]:
plt.semilogx(history2.history["lr"], history2.history["loss"])
plt.axis([1e-6, 1e-1, 0, 30])

## Model Prediction and Evaluate

In [ ]:
'''Evaluate model on Validation Dataset'''
val_perform={}
model.load_weights("muli_LSTM_check_best_lr.h5")
Bi_lstm.load_weights('muli_Bi_LSTM_check.h5'), 
conv_lstm.load_weights('muli_Conv_LSTM_check.h5')


val_perform['LSTM'] = model.evaluate(dataset_val)
val_perform['Conv_LSTM'] = Bi_lstm.evaluate(dataset_val)
val_perform['Bi_LSTM'] = conv_lstm.evaluate(dataset_val)


x = np.arange(len(val_performance))
width = 0.3

metric_name = 'mean_absolute_error'
metric_index = conv_lstm.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in val_perform.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.xticks(ticks=x, labels=val_perform.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()

#### Evaluate for single model

In [ ]:
'''This custom for single MODEL'''
conv_lstm.load_weights('muli_Conv_LSTM_check.h5')
Bi_lstm.load_weights('muli_Bi_LSTM_check.h5')
model.load_weights("muli_LSTM_check_best_lr.h5")
result=[]
for batch in test_df_data:
    predict=model.predict(batch)
    result.extend(predict)

In [ ]:
print(len(result))

In [ ]:
result_df=pd.DataFrame(result)
sns.displot(result_df)
predict_result = scaler.inverse_transform(result_df)

In [ ]:
sns.displot(predict_result) 

In [ ]:
np.max(predict_result[100:1000])

#### Result Evaluate for Multiple Model

In [ ]:
# model.load_weights("muli_LSTM_check.h5")
# Bi_lstm.load_weights('muli_Bi_LSTM_check.h5'), 
conv_lstm.load_weights('muli_Conv_LSTM_check.h5')
all_model=[ model,Bi_lstm, 
           conv_lstm ]

all_prediction=[]
for i in range(3): 
    result=[]
    for batch in test_df_data:
        predict=all_model[i].predict(batch)
        result.extend(predict)
    all_prediction.append(result)

In [ ]:
result=np.array(all_prediction)
result1=result[0, :, :]
result2=result[1, :, :]
result3=result[2, :, :]
#print(result3[1:50])

In [ ]:

col_name=['lstm', 'bi_lstm', 'conv_lstm']
result_df=pd.DataFrame( columns=col_name)
result_df['lstm']=result1.reshape(6045,)
result_df['bi_lstm']=result2.reshape(6045,)
result_df['conv_lstm']=result3.reshape(6045,)

In [ ]:
sns.displot(result_df['lstm'])
sns.displot(result_df['bi_lstm'])
sns.displot(result_df['conv_lstm'])

In [ ]:
result_df

In [ ]:
all_df=pd.DataFrame(columns=col_name)
print(all_df)
#model_=['lstm', 'bi_lstm','conv_lstm' ]
for i in range(3): 
    model_=['lstm', 'bi_lstm','conv_lstm' ]
    test=result_df[model_[i]]
    test=np.array(test)
    
    predict_result = scaler.inverse_transform(test.reshape(6045, 1))
    df=pd.DataFrame(predict_result)
    df.to_csv(model_[i]+".csv")

#     test_id=model[i]
#     all_df[test_id]=DF
    
# sns.displot(all_df['lstm']) 
# sns.displot(all_df['bi_lstm']) 
# sns.displot(all_df['conv_lstm']) 

# all_df.to_csv('predict9.csv')

In [ ]:
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

In [ ]:

for x, y in dataset_val.take(5):
    print(x.shape)
    #print(y.shape)
    result=model.predict(x)[0][0]
    print(result.shape)
    
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), model.predict(x)[0][0]],
        12,
        "Single Step Prediction",
    )

In [ ]:
predict.shape